### Download the kaggle data

To do that, you can either download the data from kaggle and put it there, or you can download the kaggle.json from your account and use these lines.

In [ ]:
!pip install kaggle # Install the Kaggle library
!mkdir ~/.kaggle # Make a directory named “.kaggle”
!cp kaggle.json ~/.kaggle/kaggle.json # Copy the “kaggle.json” into this new directory
!chmod 600 ~/.kaggle/kaggle.json # Allocate the required permission for this file.
!kaggle competitions download -c ima205-challenge-2022
# Unzip the file
!unzip -qq ima205-challenge-2022.zip

### Let's load the model and dataset

In [ ]:
import pandas as pd
import time
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import json
import torch
from torch import optim
from tqdm import tqdm
import os

from dataset import MelanomaImageDataset, MelanomaImageDatasetTest
from model import MelanomaModel

### Load some json file and the csv dataframe

In [ ]:
# Json file
with open("id_to_class.json","r") as f:
    id_to_class = json.load(f)
with open("classes_weights.json","r") as f: 
    class_weights = json.load(f)
# Link between the id of the class and its weight
id_to_weight = {
    int(k)-1 : class_weights[v] for k,v in id_to_class.items()
}
# Dataframe
df = pd.read_csv("metadataTrain.csv")

### Create dataset and dataloader

In [ ]:
# Params
image_size = 256
batch_size = 32

# paths and labels
paths = list(df["ID"])
labels = list(df["CLASS"])

# Val vs train
val_size = int(0.8*len(paths))
X_train, y_train, X_test, y_test = paths[:val_size], labels[:val_size], paths[val_size:], labels[val_size:]



# Transformations (resize and some data augmentation)
transform_train = transforms.Compose([
                                transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
                                transforms.RandomRotation(90),      # rotate +/- 10 degrees
                                transforms.RandomHorizontalFlip(),  # reverse 50% of images
                                transforms.Resize((image_size,image_size)),             # resize shortest side to 256 pixels
                                transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                    [0.229, 0.224, 0.225])
                                ])

transform_val = transforms.Compose([
                                transforms.Resize((image_size,image_size)),             # resize shortest side to 256 pixels
                                transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                    [0.229, 0.224, 0.225])
                                ])
# Create the dataset
# train
dataset_train = MelanomaImageDataset(X_train, 
                               y_train, 
                               "./Train/Train", 
                               transform_train)
# val
dataset_val = MelanomaImageDataset(X_test, 
                               y_test, 
                               "./Train/Train", 
                               transform_val)
# Deduce the dataloader
# train
dataloader_train = DataLoader(dataset_train,
                        batch_size=batch_size,
                        shuffle=True
                        )
# val
dataloader_val = DataLoader(dataset_val,
                        batch_size=batch_size,
                        shuffle=True
                        )

### Visualize some data

They do not correspond to the original images as they have been transform by some operations such as normalization and crops.

In [ ]:
plt.figure(figsize=(20,10))
for x, y in dataloader_train :
  for i in range(x.shape[0]) :
    plt.subplot(4,8,i+1)
    plt.imshow(np.transpose(x[i],(1,2,0)))
    plt.axis('off')
    plt.title(id_to_class[str(y[i].item()+1)])
  break
plt.show()

### Let's define a model

### Efficientnet b7

In [ ]:
import torchvision.models as models

freeze = False
device = torch.device("cpu" if not torch.cuda.is_available() else "cuda")
mod = models.efficientnet_b7(pretrained=True) # efficientnet_b7, resnet18, resnext101_32x8d
for param in mod.parameters() :
    param.requires_grad = freeze 
    
mod.classifier[1] = torch.nn.Linear(2560,8)
mod.features[-2].requires_grad = True
mod.features[-1].requires_grad = True

mod = mod.to(device)

print("Working on {}".format(device))

**Resnet 101 pretrained**

In [ ]:
import torchvision.models as models
freeze = False
device = torch.device("cpu" if not torch.cuda.is_available() else "cuda")
mod = models.resnext101_32x8d(pretrained=True) # efficientnet_b7, resnet18, resnext101_32x8d
"""for param in mod.parameters() :
    param.requires_grad = freeze 
mod.layer4.requires_grad = True"""
mod.fc = torch.nn.Linear(2048,8) # 2048 for resnet101, 512 for resnet18
mod = mod.to(device)

optimizer = optim.Adam(
    [
        {"params": mod.fc.parameters(), "lr": 1e-3},
        {"params": mod.layer4.parameters(), "lr": 1e-3},
        {"params": mod.layer3.parameters(), "lr": 1e-4},
        {"params": mod.layer2.parameters(), "lr": 1e-5},
        {"params": mod.layer1.parameters(), "lr": 1e-6},
        # {"params" : mod.conv1.parameters(), "lr" : 1e-6}
    ],
    lr=1e-5,
)

print("Working on {}".format(device))

Parameters for training

In [ ]:
num_epochs = 15
#to reduce the learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
# weighted loss
loss = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.8005531, 0.28592265, 0.95261733, 1.94804147, 1.20674543, 8.19375, 8.56547619, 4.04219745]).to(device))

Let's train.

In [ ]:
print("########## TRAINING ##########")
time.sleep(.8)
# logs to monitor training afterwards
logs = []
if not os.path.exists("output") :
    os.mkdir("output")
every = 2

for epoch in range(num_epochs) :
    # Training part
    items = 0
    total_loss = 0
    correct = 0
    items_w = 0
    correct_w = 0
    pbar = tqdm(enumerate(dataloader_train),total=len(dataloader_train),leave=False)
    for idx, (x, y) in pbar :
        # To device
        x, y = x.to(device), y.to(device)
        # Prediction
        out = mod(x)
        # Compute the loss
        l = loss(out, y)
        # Optimize
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        # Loss to display
        total_loss += l.detach().item()
        items += x.shape[0]
        # Metric (accuray)
        correct += (torch.argmax(out,axis=1) - y == 0).sum().detach().item()
        # Weighted accuracy
        for pre, y_p in zip((torch.argmax(out,axis=1) - y == 0), y) :
            items_w += id_to_weight[y_p.item()]
            if pre :
                correct_w += id_to_weight[y_p.item()]

        # Display
        pbar.set_postfix({
                        "Epoch" : epoch,
                        "Loss" : total_loss / items,
                        "Accuracy" : correct * 100 / items,
                        "Weighted accuracy" : correct_w * 100 / items_w
        })

        # logs
        logs.append({
            "step" : "training",
            "epoch" : epoch,
            "iteration" : idx,
            "total_loss" : total_loss / items,
            "Weighted accuracy" : correct_w * 100 / items_w,
            "lr" : scheduler.get_last_lr()[0]
        })
        with open(os.path.join("output","logs.json"), "w") as f :
          json.dump(logs,f)
    scheduler.step()

    # validation
    items = 0
    total_loss = 0
    correct = 0
    items_w = 0
    correct_w = 0
    pbar = tqdm(enumerate(dataloader_val),total=len(dataloader_val),leave=False)
    for idx, (x, y) in pbar :
        with torch.no_grad() :
            # To device
            x, y = x.to(device), y.to(device)
            # Prediction
            out = mod(x)
            # Compute the loss
            l = loss(out, y)
            # Loss to display
            total_loss += l.detach().item()
            items += x.shape[0]
            # Metric (accuray)
            correct += (torch.argmax(out,axis=1) - y == 0).sum().detach().item()
            # Weighted accuracy
            for pre, y_p in zip((torch.argmax(out,axis=1) - y == 0), y) :
                items_w += id_to_weight[y_p.item()]
                if pre :
                    correct_w += id_to_weight[y_p.item()]

            # Display
            pbar.set_postfix({
                            "Epoch" : epoch,
                            "Loss" : total_loss / items,
                            "Accuracy" : correct * 100 / items,
                            "Weighted accuracy" : correct_w * 100 / items_w
            })

            # logs
            logs.append({
                "step" : "validation",
                "epoch" : epoch,
                "iteration" : idx,
                "total_loss" : total_loss / items,
                "Weighted accuracy" : correct_w * 100 / items_w,
                "lr" : scheduler.get_last_lr()[0]
            })
            with open(os.path.join("output","logs.json"), "w") as f :
              json.dump(logs,f)

            # Scheduler 
            # scheduler.step(correct_w * 100 / items_w)
    if epoch % every == 0 :
        torch.save(mod, os.path.join("output","model_{}_epoch.pth".format(epoch)))

### Plot the learning curves

In [ ]:
training_loss = [i["total_loss"] for i in logs if i["step"] == "training" and i["iteration"] == len(dataloader_train)-1]
validation_loss = [i["total_loss"] for i in logs if i["step"] == "validation"  and i["iteration"] == len(dataloader_val)-1]

training_w_acc = [i["Weighted accuracy"] for i in logs if i["step"] == "training" and i["iteration"] == len(dataloader_train)-1]
validation_w_acc = [i["Weighted accuracy"] for i in logs if i["step"] == "validation"  and i["iteration"] == len(dataloader_val)-1]


plt.plot(np.arange(len(training_loss)),training_loss, color="blue",label="Training loss")
plt.plot(np.arange(len(validation_loss)),validation_loss, color="red",label="Validation loss")
plt.title("Losses")
plt.ylim(0,max(training_loss)*1.2)
plt.legend()
plt.show()

plt.plot(np.arange(len(training_w_acc)),training_w_acc, color="blue",label="Training accuracy")
plt.plot(np.arange(len(validation_w_acc)),validation_w_acc, color="red",label="Validation accuracy")
plt.title("Weighted accuracy")
plt.ylim(0,max(training_w_acc)*1.2)
plt.legend()
plt.show()

print("Best accuracy : {} at epoch {}".format(max(validation_w_acc), validation_w_acc.index(max(validation_w_acc))))

### Compute the score obtained based on the Kaggle weights

In [ ]:
# Without model.eval() to compare

# Do it on the validation dataloader
weights =  [0.7005531, 0.24592265, 0.95261733, 3.64804147, 1.20674543, 13.19375, 12.56547619, 5.04219745]

score_final = 0
perfect_score = 0

pbar = tqdm(enumerate(dataloader_val),total=len(dataloader_val),leave=True)
for idx, (x, y) in pbar :
    with torch.no_grad() :
        # To device
        x, y = x.to(device), y.to(device)
        # Prediction
        out = mod(x)
        # Weighted accuracy
        for pre, y_p in zip((torch.argmax(out,axis=1) - y == 0), y) :
            perfect_score += weights[y_p.item()]
            if pre :
                score_final += weights[y_p.item()]

"Your score on the validation set is {:.2f}.".format(score_final / perfect_score)

In [ ]:
mod.eval()

# Do it on the validation dataloader
weights =  [0.7005531, 0.24592265, 0.95261733, 3.64804147, 1.20674543, 13.19375, 12.56547619, 5.04219745]

score_final = 0
perfect_score = 0

pbar = tqdm(enumerate(dataloader_val),total=len(dataloader_val),leave=True)
for idx, (x, y) in pbar :
    with torch.no_grad() :
        # To device
        x, y = x.to(device), y.to(device)
        # Prediction
        out = mod(x)
        # Weighted accuracy
        for pre, y_p in zip((torch.argmax(out,axis=1) - y == 0), y) :
            perfect_score += weights[y_p.item()]
            if pre :
                score_final += weights[y_p.item()]

"Your score on the validation set is {:.2f}.".format(score_final / perfect_score)

### Prediction per class

In [ ]:
# Have access to the distribution of the predictions of the validation set

per_class = {
    i : {
        "correct" : 0,
         "seen" : 0
    }
     for i in range(8)
}

pbar = tqdm(enumerate(dataloader_val),total=len(dataloader_val),leave=True)
for idx, (x, y) in pbar :
    with torch.no_grad() :
        # To device
        x, y = x.to(device), y.to(device)
        # Prediction
        out = mod(x)
        # Weighted accuracy
        for pre, y_p in zip((torch.argmax(out,axis=1) - y == 0), y) :
            per_class[y_p.item()]["seen"] += 1
            if pre :
                per_class[y_p.item()]["correct"] += 1

for k, v in per_class.items() :
  per_class[k]["percentage"] = v["correct"] / v["seen"]
class_good_pred = {
    v : per_class[int(k)-1]["percentage"]*100 for k, v in id_to_class.items()
}

### Display the percentage of correct predictions per class, and the repartition of the class

We want the accuracy to be very similar among all classes.

In [ ]:
plt.figure(figsize=(10,10))
plt.bar(class_good_pred.keys(), list(class_good_pred.values()), color='g',label="Prediction correct", align="edge", width=0.5)
plt.bar(class_weights.keys(), list(class_weights.values()), color='b', label="Repartition", align="edge", width=-0.5)
plt.title("Class repartition along with the percentage of predictions")
plt.xticks(rotation=90,fontsize=15)
plt.legend()
plt.show()

### Perform the prediction on the test set

In [ ]:
# Dataframe
df = pd.read_csv("metadataTest.csv")

# Params
image_size = 256
batch_size = 16

paths = list(df["ID"])

# Transformations (resize and some data augmentation)
transform_test = transforms.Compose([
                                transforms.Resize((image_size,image_size)),             # resize shortest side to 256 pixels
                                transforms.CenterCrop(224),         # crop longest side to 300 pixels at center
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                    [0.229, 0.224, 0.225])
                                ])
# Create the dataset
# test
dataset_test = MelanomaImageDatasetTest(paths, 
                               "./Test/Test", 
                               transform_test)
# Deduce the dataloader
# test
dataloader_test = DataLoader(dataset_test,
                        batch_size=batch_size,
                        shuffle=True
                        )

In [ ]:
# Perform the predictions
class_pred = []
ids = []

for x, y in tqdm(dataloader_test) :
  x = x.to(device)
  out = mod(x)
  out = torch.argmax(out,axis=1).cpu().detach().numpy()
  for cl, id in zip(out, y) :
    class_pred.append(cl+1)
    ids.append(id)

In [ ]:
# Show repartition of preds
all_preds = {
    k : 0 for v,k in id_to_class.items()
}

for pr in class_pred :
    all_preds[id_to_class[str(pr)]] += 1 / len(class_pred) * 100

# Calculate the supposed repartition
weights =  [0.7005531, 0.24592265, 0.95261733, 3.64804147, 1.20674543, 13.19375, 12.56547619, 5.04219745]
actual_rep = {
    k : 1/ weights[idx] * 10 for idx, k in enumerate(all_preds.keys())
}

plt.figure(figsize=(10,10))
plt.bar(all_preds.keys(), list(all_preds.values()), color='g',label="Prediction Repartition", align="edge", width=0.5)
plt.bar(actual_rep.keys(), list(actual_rep.values()), color='b', label="Actual Repartition", align="edge", width=-0.5)
plt.xticks(rotation=90,fontsize=15)
plt.legend()
plt.show()

In [ ]:
# Store the result in a dataframe in order to submit afterwards
submission_df = pd.DataFrame()
submission_df["ID"] = ids
submission_df["CLASS"] = class_pred

submission_df.to_csv("submission.csv", index=False)